In [1]:
import numpy as np
import sciopy as sc
from .sciopy.setup_m import GeneralSystemMessages, SoftwareReset, SetLED_Mode

ImportError: attempted relative import with no known parent package

In [2]:
ser = sc.connect_COM_port()

Connection to COM3 is established.


In [3]:
GeneralSystemMessages?

Object `GeneralSystemMessages` not found.


In [5]:
bytearray([0xD2, 0x00, 0xD2])

bytearray(b'\xd2\x00\xd2')